In [1]:
import ast
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("merged_cns_maureen_annotations.csv")
df.label = df.label.apply(lambda x: ast.literal_eval(x)[:-1])
df.classifications.fillna("nan", inplace=True)

<ipython-input-2-f95703b8c1fc>:1: DtypeWarning: Columns (0,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_cns_maureen_annotations.csv")


In [4]:
# Add label indicator
attr = df.columns[-20:-2]
df["label_indicator"] = df[attr].apply(lambda x: 1 if x.sum() > 0 else 0, axis=1)
df["label_count"] = df[attr].apply(lambda x: x.sum(), axis=1)
df["full_label_count"] = df.label.apply(lambda x: sum(x))

In [5]:
# Count species classifications
def count_blanks(x):
    count = 0
    for i in x:
        if i == "blank":
            count += 1
    return count


def count_chimps(x):
    count = 0
    for i in x:
        if i == "chimpanzee" or i == "chimp":
            count += 1
            try:
                split_i = i.split("/")
                if "chimpanzee" or "chimp" in split_i:
                    count += 1
            except:
                continue
    return count


df["split_classifications"] = df.classifications.apply(lambda x: x.lower().split(","))
df["blank_count"] = df.split_classifications.apply(lambda x: count_blanks(x))
df["classifications_count"] = df.split_classifications.apply(lambda x: len(x))
df["blank_ratio"] = df.blank_count / df.classifications_count * 100

**Videos which have been labelled but are empty!**

In [6]:
# These will likely need to be removed but check beforehand
empty_labelled_videos = df[
    (df.behavioral_context == "False")
    & (df["max"].isna())
    & (df["min"].isna())
    & (df.blank_ratio == 100.0)
    & (df.label_indicator == 1)
][
    ["subject_id", "video_id", "behavioral_context", "label_count", "max", "min"]
    + list(attr)
].video_id.values.tolist()

# These will likely need to be removed but check beforehand
empty_labelled_videos.extend(
    df[(df["max"].isna()) & (df["min"].isna()) & (df.label_indicator == 1)][
        ["subject_id", "video_id", "behavioral_context", "label_count", "max", "min"]
        + list(attr)
    ].video_id.values.tolist()
)

In [7]:
set(empty_labelled_videos)

{'djo_cam05_0697652_0598620_20121226_pict0087',
 'djo_cam10_0698763_0597847_20130109_ek000008',
 'nim_vid01_0545838_0820109_20130923_ek000228',
 'tair_cam15_687357_648673_20131220_pict0313',
 'tair_cam15_687357_648673_20131220_pict0368',
 'tair_cam1_687372_648736_20131223_pict0678',
 'tair_cam22_688836_647457_20131025_ek000246',
 'tair_cam24_685978_649225_20131123_pict0271',
 'tair_cam25_688407_650454_20130611_pict0377',
 'tair_cam28_688388_651361_20130625_ek000290',
 'tair_cam30_687126_648216_20130904_ek000287',
 'tair_cam31_686481_649929_20131209_ek000529'}

**Videos not labelled that should be!**

In [8]:
df.behavioral_context.unique()

array(['travel', 'climbing', 'resting', 'displaying', 'feeding',
       'unclear', 'camera reaction', 'aggression', 'tool use', 'playing',
       'greeting', 'grooming', 'sexual', 'tool use algae', 'reassurance',
       'tool use termites', 'tool use nuts', 'False',
       'tool use stone throwing'], dtype=object)

In [9]:
df.camera_reaction_mm.replace("ues", True, inplace=True)
df.camera_reaction_mm.replace("False", False, inplace=True)
df.camera_reaction_mm.replace("True", True, inplace=True)
df.camera_reaction_mm.value_counts()

df.tool_use_mm.replace("ues", True, inplace=True)
df.tool_use_mm.replace("False", False, inplace=True)
df.tool_use_mm.replace("True", True, inplace=True)
df.tool_use_mm.value_counts()

df.vocalization_mm.replace("ues", True, inplace=True)
df.vocalization_mm.replace("False", False, inplace=True)
df.vocalization_mm.replace("offscreen", False, inplace=True)
df.vocalization_mm.replace("True", True, inplace=True)
df.vocalization_mm.value_counts()

df.bipedal_mm.replace("ues", True, inplace=True)
df.bipedal_mm.replace("False", False, inplace=True)
df.bipedal_mm.replace("True", True, inplace=True)
df.bipedal_mm.value_counts()

bipedal_mm
False    53883
True      1364
Name: count, dtype: int64

In [10]:
unlabelled_videos = df[(df.behavioral_context != "False") & (df.full_label_count == 0)]

In [11]:
# Process behavioural context

unlabelled_videos.p_travel = unlabelled_videos.apply(
    lambda x: True
    if ((x["behavioral_context"] == "travel") & (x["p_travel"] == False))
    else False,
    axis=1,
)

unlabelled_videos.p_climbing = unlabelled_videos.apply(
    lambda x: True
    if ((x["behavioral_context"] == "climbing") & (x["p_climbing"] == False))
    else False,
    axis=1,
)

unlabelled_videos.p_feeding = unlabelled_videos.apply(
    lambda x: True
    if ((x["behavioral_context"] == "feeding") & (x["p_feeding"] == False))
    else False,
    axis=1,
)

unlabelled_videos.p_resting = unlabelled_videos.apply(
    lambda x: True
    if ((x["behavioral_context"] == "resting") & (x["p_resting"] == False))
    else False,
    axis=1,
)

unlabelled_videos.p_tool_use = unlabelled_videos.apply(
    lambda x: True
    if ((x["behavioral_context"] == "tool use") & (x["p_tool_use"] == False))
    else False,
    axis=1,
)

unlabelled_videos.p_camera_reaction = unlabelled_videos.apply(
    lambda x: True
    if (
        (x["behavioral_context"] == "camera reaction")
        & (x["p_camera_reaction"] == False)
    )
    else False,
    axis=1,
)

<ipython-input-11-501e014a1fc9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabelled_videos.p_travel = unlabelled_videos.apply(
<ipython-input-11-501e014a1fc9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabelled_videos.p_climbing = unlabelled_videos.apply(
<ipython-input-11-501e014a1fc9>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
# Process camera reaction, tool use, vocalization, bipedal
unlabelled_videos.p_camera_reaction = unlabelled_videos.apply(
    lambda x: True
    if (
        ((x["camera_reaction_mm"] == True) & (x["p_camera_reaction"] == False))
        | ((x["camera_reaction_mm"] == True) & (x["p_camera_reaction"] == True))
    )
    else False,
    axis=1,
)

unlabelled_videos.p_tool_use = unlabelled_videos.apply(
    lambda x: True
    if (
        ((x["tool_use_mm"] == True) & (x["p_tool_use"] == False))
        | ((x["tool_use_mm"] == True) & (x["p_tool_use"] == True))
    )
    else False,
    axis=1,
)

unlabelled_videos.p_vocalization = unlabelled_videos.apply(
    lambda x: True
    if (
        ((x["vocalization_mm"] == True) & (x["p_vocalisation"] == False))
        | ((x["vocalization_mm"] == True) & (x["p_vocalisation"] == True))
    )
    else False,
    axis=1,
)

unlabelled_videos.p_bipedal = unlabelled_videos.apply(
    lambda x: True
    if (
        ((x["bipedal_mm"] == True) & (x["p_bipedal"] == False))
        | ((x["bipedal_mm"] == True) & (x["p_bipedal"] == False))
    )
    else False,
    axis=1,
)

In [17]:
# Removing these videos for now since they are highly ambiguous
unlabelled_videos.video_id.nunique()

90

In [22]:
# Videos to remove
pd.DataFrame(
    {
        "videos_to_remove": list(
            set(empty_labelled_videos + unlabelled_videos.video_id.values.tolist())
        )
    }
).to_csv("videos_to_remove.csv", index=False)

102

In [ ]:
unique_species_classifications = []
df["split_classifications"] = df.classifications.apply(lambda x: x.lower().split(","))
for row in df.split_classifications.values:
    for i in row:
        unique_species_classifications.append(i)
unique_species_classifications = list(set(unique_species_classifications))

In [ ]:
# conclusion - blank is only ever used alone...
blank_categories = []
for item in unique_species_classifications:
    if item == "blank":
        blank_categories.append(item)
    else:
        try:
            split_item = item.split("/")
            for i in split_item:
                if i == "blank":
                    blank_categories.append(item)
        except:
            continue
blank_categories

**Eliminate bogus classifications**

In [ ]:
unique_species_classifications